In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 17:19:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
#!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2023-02-28 20:09:30--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.36.104, 52.217.234.128, 52.217.236.160, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.36.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.2’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2023-02-28 20:09:31 (34.4 MB/s) - ‘taxi+_zone_lookup.csv.2’ saved [12322/12322]



In [15]:
df_taxi = spark.read \
    .option("header", "true") \
    .option("inferSchema",True)\
    .csv('taxi+_zone_lookup.csv')

In [16]:
df_taxi.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [17]:
df_taxi.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [18]:
df_taxi.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [19]:
df = spark.read\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv("fhv_tripdata_2019-12.csv.gz")

In [20]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [21]:
from pyspark.sql import functions as F

In [22]:
string_udf = F.udf(lambda x: str(x))

In [23]:
df.withColumn("str_pu_id",string_udf(df.PUlocationID)).show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|str_pu_id|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+
|              B00009|2019-12-01 00:47:00|2019-12-01 00:52:00|         264|         264|   null|                B00009|      264|
|              B00009|2019-12-01 00:27:00|2019-12-01 00:44:00|         264|         264|   null|                B00009|      264|
|              B00014|2019-12-01 00:42:18|2019-12-01 01:31:31|         264|         264|   null|                B00014|      264|
|              B00014|2019-12-01 00:43:08|2019-12-01 01:07:38|         264|         264|   null|                B00014|      264|
|     B00021         |2019-12-01 00:52:19|2019-12-01 00:59:39|          56|          56|  

In [24]:
df.rdd.getNumPartitions()

1

In [25]:
#df.repartition(4).write.parquet("test_fhv_12")

AnalysisException: path file:/Users/soapycat/Documents/Persona/DataZooCamp/week_5_spark/test_fhv_12 already exists.

In [26]:
df_12 = spark.read.parquet('test_fhv_12/*')

In [27]:
df_6_before = spark.read.option("header",True)\
    .option("inferSchema",True)\
    .csv('fhv_tripdata_2019-06.csv.gz')

In [ ]:
df_6_before.repartition(4).write.parquet("test_fhv_6")

In [28]:
df_6 = spark.read.parquet('test_fhv_6/*')

In [29]:
df_6.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [30]:
common_cols = list(set(df_6.columns) & set(df_12.columns))

In [31]:
df_6_sel = df_6.select(common_cols).withColumn("month",F.lit("June"))

In [32]:
df_12_sel = df_12.select(common_cols).withColumn("month",F.lit("December"))

In [33]:
df_fhv_all = df_6_sel.unionAll(df_12_sel)

In [34]:
df_fhv_all.groupBy('month').count().show()

+--------+-------+
|   month|  count|
+--------+-------+
|    June|2009886|
|December|2044196|
+--------+-------+



In [35]:
df_fhv_all.createOrReplaceTempView("fhv")

In [36]:
sql_result = spark.sql("""
    select month, count(1) from fhv group by 1
""")

sql_result.show()

+--------+--------+
|   month|count(1)|
+--------+--------+
|    June| 2009886|
|December| 2044196|
+--------+--------+



In [37]:
sql_result.show()

+--------+--------+
|   month|count(1)|
+--------+--------+
|    June| 2009886|
|December| 2044196|
+--------+--------+



In [41]:
sql_result.rdd.getNumPartitions()

1

In [43]:
df_fhv_all.show(5)

+--------------------+----------------------+-------------------+-------------------+-------+------------+------------+-----+
|dispatching_base_num|Affiliated_base_number|    pickup_datetime|   dropOff_datetime|SR_Flag|PUlocationID|DOlocationID|month|
+--------------------+----------------------+-------------------+-------------------+-------+------------+------------+-----+
|              B01437|                B02872|2019-06-20 22:29:42|2019-06-20 22:35:06|   null|         264|         265| June|
|              B01087|                B01087|2019-06-17 19:03:43|2019-06-17 20:20:43|   null|          65|          55| June|
|              B00900|                B00900|2019-06-05 16:19:02|2019-06-05 16:22:20|   null|         264|         265| June|
|              B01469|                B02867|2019-06-26 09:11:35|2019-06-26 09:22:23|   null|         264|         265| June|
|              B01432|                B01432|2019-06-23 19:02:35|2019-06-23 19:14:38|   null|         264|         264

In [46]:
df_taxi.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [49]:
df_fhv_all.join(df_taxi
                , on = df_fhv_all.PUlocationID == df_taxi.LocationID
                , how='left').show(5)

+--------------------+----------------------+-------------------+-------------------+-------+------------+------------+-----+----------+--------+--------------------+------------+
|dispatching_base_num|Affiliated_base_number|    pickup_datetime|   dropOff_datetime|SR_Flag|PUlocationID|DOlocationID|month|LocationID| Borough|                Zone|service_zone|
+--------------------+----------------------+-------------------+-------------------+-------+------------+------------+-----+----------+--------+--------------------+------------+
|              B01437|                B02872|2019-06-20 22:29:42|2019-06-20 22:35:06|   null|         264|         265| June|       264| Unknown|                  NV|         N/A|
|              B01087|                B01087|2019-06-17 19:03:43|2019-06-17 20:20:43|   null|          65|          55| June|        65|Brooklyn|Downtown Brooklyn...|   Boro Zone|
|              B00900|                B00900|2019-06-05 16:19:02|2019-06-05 16:22:20|   null|       

In [51]:
df_green = spark.read\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .parquet("green_tripdata_2019-01.parquet")

In [54]:
rdd_green = df_green\
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount')\
    .rdd

In [60]:
rdd_green.take(10)

[Row(lpep_pickup_datetime=datetime.datetime(2018, 12, 21, 22, 17, 29), PULocationID=264, total_amount=4.3),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 10, 16), PULocationID=97, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 27, 11), PULocationID=49, total_amount=5.8),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 46, 20), PULocationID=189, total_amount=19.71),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 19, 6), PULocationID=82, total_amount=19.3),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 12, 35), PULocationID=49, total_amount=7.8),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 47, 55), PULocationID=255, total_amount=14.8),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 12, 47), PULocationID=76, total_amount=17.3),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 1, 7, 16, 23), PULocationID=25, total_amount=26.8),
 Row(lpep_pickup_datetime=datetime.datetime(2019, 1, 

In [64]:
rdd_green.count()

630918

In [70]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [71]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [73]:
from collections import namedtuple

RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [75]:
rdd_result = rdd_green\
    .filter(lambda row: row.total_amount > 10)\
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap)

In [77]:
rdd_result.take(5)

[RevenueRow(hour=datetime.datetime(2019, 1, 1, 7, 0), zone=76, revenue=256.92, count=8),
 RevenueRow(hour=datetime.datetime(2019, 1, 1, 7, 0), zone=85, revenue=131.46, count=4),
 RevenueRow(hour=datetime.datetime(2019, 1, 1, 7, 0), zone=146, revenue=49.91, count=3),
 RevenueRow(hour=datetime.datetime(2019, 1, 1, 7, 0), zone=181, revenue=1709.5899999999992, count=57),
 RevenueRow(hour=datetime.datetime(2019, 1, 1, 7, 0), zone=41, revenue=988.6899999999993, count=55)]

In [79]:
df_result = rdd_result.toDF()

In [80]:
df_result.show(5)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2019-01-01 07:00:00|  76|            256.92|    8|
|2019-01-01 07:00:00|  85|            131.46|    4|
|2019-01-01 07:00:00| 146|             49.91|    3|
|2019-01-01 07:00:00| 181|1709.5899999999992|   57|
|2019-01-01 07:00:00|  41| 988.6899999999993|   55|
+-------------------+----+------------------+-----+
only showing top 5 rows

